In [4]:
import warnings
warnings.filterwarnings('ignore') # Filter out warnings
import numpy as np
import pandas as pd
import random as rnd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
from scipy import stats
import math
from math import isnan
import time
from collections import Counter

## Helper function

In [5]:
## SUBJECT TO CHANGE... COLUMNS ARE ADAPTIVE
# Function of getting history for all orders
def GetOrderHistory(df_original):
    df = df_original.copy(deep=False) 
    t = time.time()
    table = {}
    for index, row in df.iterrows():
        if (row['OrderID']!=0):   # don't need records where orderID==0; remove row['Type'], row['Dir'] is okay?
            d = {row['Time_stamp']: [ row['V'],row['P'],row['Label'],row['Level'],\
                                      row['Volts'],row['N_OrderSubmit'],row['bid-ask spread 1'],\
                                      row['Submit_rate'],row['Cancel_rate'],row['Execute_rate'] 
                                    ]}
            if (row['OrderID'] not in table.keys()):
                li = []
                li.append(d)
                table[row['OrderID']] = li
            else:
                table[row['OrderID']].append(d)
    print("Completed. It takes about {0:.2f} minutes.".format((time.time()-t)/60))        
    return table

In [6]:
def ReturnTwoStates_orders(df_original,type_list,orderhistory):
    df = df_original.copy(deep=False)
    t = time.time()
    TwoStates = {}
    # only want records that are of those types, and OrderID != 0
    df = df.loc[df['Type'].isin(type_list)]
    Ids = set(df['OrderID'].tolist())
    Ids.discard(0)
    # look through all order ids in the dictionary
    for k, v in orderhistory.items():
        if k in Ids:
            temp_v = orderhistory[k]
            # avoid cases where there is only one event happened to a single order
            if (len(temp_v)>=2):
                # get first event and the very last event
                st_ed = temp_v[::len(temp_v)-1]
                TwoStates[k] = st_ed
    print("Completed. It takes about {0:.2f} minutes.".format((time.time()-t)/60))  
    return TwoStates

In [7]:
def HistoryToDF_2(history_dict):
    t = time.time()
    colnames = ['OrderID','Time','V','P','Label','Level',\
                'Volts','N_OrderSubmit','Spread','Submit_rate','Cancel_rate','Execute_rate']
    twoStatesName = ['_st','_ed']
    cols = [colnames[0]]
    for i in range(len(twoStatesName)):
        for j in range(1,len(colnames)):
            cols.append(colnames[j]+twoStatesName[i])
    df = pd.DataFrame(columns=cols)
    # traverse through every single orderID
    ct = 0 
    totalItems = len(history_dict.keys())
    steps = int(totalItems/10)
    for k, v in history_dict.items():
        new_row = []
        new_row.append(k)
        new_row.append(list(v[0].keys())[0])
        new_row.extend(list(v[0].values())[0])
        new_row.append(list(v[-1].keys())[0])
        new_row.extend(list(v[-1].values())[0])
        # appending to the dataframe
        df.loc[ct] = new_row
        ct += 1
        if (ct%steps==0):
            print("{0:.2f}%...".format(ct*100/totalItems))
    print("Completed. It takes about {0:.2f} minutes.".format((time.time()-t)/60))
    return df

In [8]:
def HistoryToCSV(history_dict,file_name):
    t = time.time()
    import csv
    colnames = ['OrderID','Time','V','P','Label','Level',\
                'Volts','N_OrderSubmit','Spread','Submit_rate','Cancel_rate','Execute_rate']
    twoStatesName = ['_st','_ed']
    header = [colnames[0]]
    for i in range(len(twoStatesName)):
        for j in range(1,len(colnames)):
            header.append(colnames[j]+twoStatesName[i])
    # initialize a writer
    with open(file_name,'w',newline="") as f:
        writer = csv.writer(f)
        # write heeder row first
        #writer.writerows(header)
        writer.writerows([header])
        # traverse through every single orderID
        ct = 0 
        totalItems = len(history_dict.keys())
        steps = int(totalItems/10)
        for k, v in history_dict.items():
            new_row =[]
            new_row.append(k)
            new_row.append(list(v[0].keys())[0])
            new_row.extend(list(v[0].values())[0])
            new_row.append(list(v[-1].keys())[0])
            new_row.extend(list(v[-1].values())[0])
            # write the new row to csv
            writer.writerows([new_row])
            ct += 1
            if (ct%steps==0):
                print("{0:.2f}%".format(ct*100/totalItems))
    print("Completed. It takes about {0:.2f} minutes.".format((time.time()-t)/60))
    return 

### Processing starts from here

## READ THE BIG CSV, CONTINUE...

In [9]:
df = pd.read_csv("Messages_10amTo4pm_AMZN.csv")
df.head(5)

Unnamed: 0    Time_stamp  Type   OrderID    V        P  Dir  \
0       13689  36000.040921     3  46494609   19  2243500   -1   
1       13690  36000.040981     1  46531222   19  2241500    1   
2       13691  36000.042313     3  46520399  100  2244400   -1   
3       13692  36000.214317     3  46531222   19  2241500    1   
4       13693  36000.214402     1  46542025   19  2243500   -1   

                            Time  P_ask_1  V_ask_1      ...       mid-price 3  \
0  2012-06-21 10:00:00.040920000  2243500      100      ...         2242650.0   
1  2012-06-21 10:00:00.040981000  2243500      100      ...         2242650.0   
2  2012-06-21 10:00:00.042313000  2243500      100      ...         2242650.0   
3  2012-06-21 10:00:00.214317000  2243500      100      ...         2242650.0   
4  2012-06-21 10:00:00.214402000  2243500      119      ...         2242650.0   

   bid-ask spread 4  mid-price 4  bid-ask spread 5  mid-price 5       Volts  \
0              3300    2242750.0              3500    2242750.0  2082.61304   
1              3300    2242750.0              3500    2242750.0  2082.61304   
2              3300    2242750.0              3500    2242750.0  2082.61304   
3              3300    2242750.0              3500    2242750.0  2082.61304   
4              3300    2242750.0              3500    2242750.0  2082.61304   

   N_OrderSubmit  Submit_rate  Cancel_rate  Execute_rate  
0           2830        50.56        43.99          5.45  
1           2831        50.57        43.98          5.45  
2           2831        50.56        43.99          5.45  
3           2831        50.55        44.00          5.45  
4           2832        50.56        43.99          5.45  

[5 rows x 61 columns]

In [10]:
print('total column number: ',len(df.columns))
df.columns

total column number:  61


Index(['Unnamed: 0', 'Time_stamp', 'Type', 'OrderID', 'V', 'P', 'Dir', 'Time',
       'P_ask_1', 'V_ask_1', 'P_bid_1', 'V_bid_1', 'P_ask_2', 'V_ask_2',
       'P_bid_2', 'V_bid_2', 'P_ask_3', 'V_ask_3', 'P_bid_3', 'V_bid_3',
       'P_ask_4', 'V_ask_4', 'P_bid_4', 'V_bid_4', 'P_ask_5', 'V_ask_5',
       'P_bid_5', 'V_bid_5', 'Label', 'Level', 'bid-ask spread 1',
       'mid-price 1', 'd_P_ask_51', 'd_P_bid_51', 'd_P_ask_21', 'd_P_bid_21',
       'd_P_ask_32', 'd_P_bid_32', 'd_P_ask_43', 'd_P_bid_43', 'd_P_ask_54',
       'd_P_bid_54', 'Mean_P_ask', 'Mean_P_bid', 'Mean_V_ask', 'Mean_V_bid',
       'P_accu', 'V_accu', 'bid-ask spread 2', 'mid-price 2',
       'bid-ask spread 3', 'mid-price 3', 'bid-ask spread 4', 'mid-price 4',
       'bid-ask spread 5', 'mid-price 5', 'Volts', 'N_OrderSubmit',
       'Submit_rate', 'Cancel_rate', 'Execute_rate'],
      dtype='object')

#### 1) GROUPING ALL MESSAGES BY ORDER IDS

In [11]:
OrderHistory = GetOrderHistory(df)

Completed. It takes about 0.59 minutes.


In [12]:
# take a look at first 1 item of OrderHistory
from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a dictionary"
    return dict(islice(iterable, n))
n_items = take(1, OrderHistory.items())
n_items   
# OrderID: [{Time_stamp:[V,P,Label,level,Volts,N_OrderSubmit,bid-ask spread 1,Submit_rate,Cancel_rate,Execute_rate]}]

{46494609: [{36000.040920531: [19,
    2243500,
    'c',
    1.0,
    2082.6130404025657,
    2830,
    1800,
    50.56,
    43.99,
    5.45]}]}

In [15]:
counts_allHistories = Counter(his_list)
counts_allHistories

Counter({1: 19003,
         2: 57673,
         3: 1369,
         4: 196,
         5: 77,
         6: 23,
         7: 12,
         8: 5,
         9: 4,
         10: 3,
         11: 4,
         13: 1,
         14: 1,
         15: 3,
         16: 1,
         18: 1})

In [16]:
his_list = []
ct_items = 0
for k,v in OrderHistory.items():
    ct_items += 1
    his_list.append(len(v))

print("total number of orders is",int(ct_items))    
print("orders with 2 events is about {0:.2f}%.".format(57673*100/78376))

total number of orders is 78376
orders with 2 events is about 73.59%.


In [17]:
ct = 0
for k, v in OrderHistory.items():
    ct += len(v)
print('average number of event for an order is {0:.3f}'.format(ct/len(OrderHistory.keys())))

average number of event for an order is 1.788


#### 2) GETTING START AND END STATES FOR SELECTED ORDER TYPES, STORE IN DICTIONARIES

In [18]:
OE_history = ReturnTwoStates_orders(df_original=df,type_list=[4,5],orderhistory=OrderHistory)

Completed. It takes about 0.00 minutes.


In [19]:
OC_history = ReturnTwoStates_orders(df_original=df,type_list=[2,3],orderhistory=OrderHistory)

Completed. It takes about 0.00 minutes.


In [21]:
print("total number of execution orders and cancellation orders are {}, {} respectively."
      .format(len(OE_history.keys()),len(OC_history.keys())))

total number of execution orders and cancellation orders are 6016, 53789 respectively.


#### deprecated: too slow. FORMAT ORDER HISTORIES INTO DATAFRAMES

In [22]:
# OE_df = HistoryToDF_2(OE_history)
# OC_df = HistoryToDF_2(OC_history)
# 4) TO CSV
# OC_df.to_csv("OC_twoStates.csv")
# OE_df.to_csv("OE_twoStates.csv")

#### 3) DIRECTLY WRITE OC AND OE HISTORY INTO CSV

In [23]:
HistoryToCSV(OE_history,file_name="Orders_execute_AMZN.csv")

9.99%
19.98%
29.97%
39.96%
49.95%
59.94%
69.93%
79.92%
89.91%
99.90%
Completed. It takes about 0.00 minutes.


In [24]:
HistoryToCSV(OC_history,file_name="Orders_cancel_AMZN.csv")

10.00%
20.00%
29.99%
39.99%
49.99%
59.99%
69.99%
79.99%
89.98%
99.98%
Completed. It takes about 0.02 minutes.


In [ ]:
#### 4) CLEANING START LEVEL AND END LEVEL

## TESTING.... 